#### Clean data about crashes where pedestrians or cyclists were injured or killed in Brooklyn
##### Remove entries without coordinates

In [1]:
import warnings, os
import pandas as pd
import numpy as np
import geopandas as gpd
%matplotlib inline

In [2]:
# Load crash data file
crashes = gpd.read_file("../Raw Data/PedestriansCyclistsKilledInjured.csv")

In [3]:
df = pd.read_csv('../Raw Data/PedestriansCyclistsKilledInjured.csv')

In [4]:
df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,"(40.623104, -73.95809)",EAST 18 STREET,AVENUE K,NaN,...,NaN,NaN,NaN,NaN,4486516,Sedan,NaN,NaN,NaN,NaN
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,"(40.687534, -73.9775)",FULTON STREET,SAINT FELIX STREET,NaN,...,Unspecified,NaN,NaN,NaN,4487052,Sedan,Bike,NaN,NaN,NaN
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,"(40.65011, -73.930214)",UTICA AVENUE,SNYDER AVENUE,NaN,...,NaN,NaN,NaN,NaN,4522136,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,"(40.59207, -73.96299)",EAST 7 STREET,CRAWFORD AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,"(40.640835, -73.98967)",12 AVENUE,41 STREET,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4485355,Sedan,Bike,NaN,NaN,NaN


In [5]:
# Keep only the specified columns
columns_to_keep = [
    'CRASH DATE',
    'CRASH TIME', 
    'BOROUGH',
    'ZIP CODE',
    'LATITUDE',
    'LONGITUDE',
    'NUMBER OF PERSONS INJURED',
    'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED',
    'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED',
    'NUMBER OF CYCLIST KILLED'
]

df_filtered = df[columns_to_keep]

In [6]:
df_filtered.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,1,0.0,1,0,0,0
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,1,0.0,0,0,1,0
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,1,0.0,1,0,0,0
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,1,0.0,0,0,1,0
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,1,0.0,0,0,1,0


In [7]:
# Remove rows with 0,0 coordinates or null values
df_filtered = df_filtered[
    (df_filtered['LATITUDE'] != 0) & 
    (df_filtered['LONGITUDE'] != 0) &
    (df_filtered['LATITUDE'].notna()) &
    (df_filtered['LONGITUDE'].notna())
]

print(f"Removed {len(df) - len(df_filtered)} rows with invalid coordinates")
print(f"Remaining rows: {len(df_filtered)}")

Removed 1131 rows with invalid coordinates
Remaining rows: 50493


#### Create GeoDataFrame

In [8]:
# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_filtered, 
    geometry=gpd.points_from_xy(df_filtered['LONGITUDE'], df_filtered['LATITUDE']),
    crs='EPSG:4326'
)

In [9]:
gdf = gdf.to_crs('EPSG:2263')

In [10]:
gdf.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,geometry
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,1,0.0,1,0,0,0,POINT (995884.243 166292.752)
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,1,0.0,0,0,1,0,POINT (990489.985 189764.409)
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,1,0.0,1,0,0,0,POINT (1003614.798 176136.698)
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,1,0.0,0,0,1,0,POINT (994528.777 154985.651)
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,1,0.0,0,0,1,0,POINT (987116.853 172750.017)


#### Create a column called CRASH_DATETIME in the datetime format

In [11]:
# Check data type of CRASH DATE column
print(df_filtered['CRASH DATE'].dtype)

object


In [12]:
# Combine the original string columns before converting
gdf['CRASH_DATETIME'] = pd.to_datetime(gdf['CRASH DATE'].astype(str) + ' ' + gdf['CRASH TIME'].astype(str))

In [13]:
print(gdf['CRASH_DATETIME'].head())
print(gdf['CRASH_DATETIME'].dtype)



0   2021-12-14 17:31:00
1   2021-12-14 12:54:00
2   2022-04-12 19:56:00
3   2021-12-09 20:20:00
4   2021-12-09 23:15:00
Name: CRASH_DATETIME, dtype: datetime64[ns]
datetime64[ns]


In [14]:
# See date range
print("Date range:")
print("Earliest:", gdf['CRASH_DATETIME'].min())
print("Latest:", gdf['CRASH_DATETIME'].max())

Date range:
Earliest: 2012-07-01 01:10:00
Latest: 2025-06-29 19:45:00


In [15]:
gdf.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,geometry,CRASH_DATETIME
0,12/14/2021,17:31,BROOKLYN,11230,40.623104,-73.958090,1,0.0,1,0,0,0,POINT (995884.243 166292.752),2021-12-14 17:31:00
1,12/14/2021,12:54,BROOKLYN,11217,40.687534,-73.977500,1,0.0,0,0,1,0,POINT (990489.985 189764.409),2021-12-14 12:54:00
2,04/12/2022,19:56,BROOKLYN,11203,40.650110,-73.930214,1,0.0,1,0,0,0,POINT (1003614.798 176136.698),2022-04-12 19:56:00
3,12/09/2021,20:20,BROOKLYN,11223,40.592070,-73.962990,1,0.0,0,0,1,0,POINT (994528.777 154985.651),2021-12-09 20:20:00
4,12/09/2021,23:15,BROOKLYN,11218,40.640835,-73.989670,1,0.0,0,0,1,0,POINT (987116.853 172750.017),2021-12-09 23:15:00


#### Creating a weekly rate for 11 meter square clusters.
##### Since most of the crashes happen at intersections anyway, this will cluster crashes at an 'intersection'

In [17]:
# Check for null values in all columns
print("Null values per column:")
print(gdf.isnull().sum())

# Show percentage of nulls
print("\nPercentage of nulls per column:")
print((gdf.isnull().sum() / len(gdf) * 100).round(2))

# Get a summary of data types and non-null counts
print("\nColumn info:")
print(gdf.info())

# Check for empty strings (sometimes appears as valid data but is actually missing)
print("\nEmpty string values per column:")
for col in gdf.columns:
    if gdf[col].dtype == 'object':  # Only check text columns
        empty_count = (gdf[col] == '').sum()
        if empty_count > 0:
            print(f"{col}: {empty_count} empty strings")

# Check for any rows that have nulls in critical columns
critical_columns = ['CRASH_DATETIME', 'LATITUDE', 'LONGITUDE', 
                   'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
                   'NUMBER OF CYCLIST KILLED', 'NUMBER OF CYCLIST INJURED']

print(f"\nRows with nulls in critical columns:")
for col in critical_columns:
    if col in gdf.columns:
        null_count = gdf[col].isnull().sum()
        print(f"{col}: {null_count} nulls")

Null values per column:
CRASH DATE                       0
CRASH TIME                       0
BOROUGH                          0
ZIP CODE                         0
LATITUDE                         0
LONGITUDE                        0
NUMBER OF PERSONS INJURED        0
NUMBER OF PERSONS KILLED         1
NUMBER OF PEDESTRIANS INJURED    0
NUMBER OF PEDESTRIANS KILLED     0
NUMBER OF CYCLIST INJURED        0
NUMBER OF CYCLIST KILLED         0
geometry                         0
CRASH_DATETIME                   0
dtype: int64

Percentage of nulls per column:
CRASH DATE                       0.0
CRASH TIME                       0.0
BOROUGH                          0.0
ZIP CODE                         0.0
LATITUDE                         0.0
LONGITUDE                        0.0
NUMBER OF PERSONS INJURED        0.0
NUMBER OF PERSONS KILLED         0.0
NUMBER OF PEDESTRIANS INJURED    0.0
NUMBER OF PEDESTRIANS KILLED     0.0
NUMBER OF CYCLIST INJURED        0.0
NUMBER OF CYCLIST KILLED         

In [18]:
# Create intersection ID (round to ~11 meter precision)
gdf['intersection_id'] = (gdf['LATITUDE'].round(4).astype(str) + '_' + 
                         gdf['LONGITUDE'].round(4).astype(str))

In [19]:
# Create year-week field
gdf['year_week'] = gdf['CRASH_DATETIME'].dt.strftime('%Y-W%U')


In [20]:
# Create total casualties column
gdf['total_casualties'] = (gdf['NUMBER OF PEDESTRIANS KILLED'] + 
                          gdf['NUMBER OF PEDESTRIANS INJURED'] + 
                          gdf['NUMBER OF CYCLIST KILLED'] + 
                          gdf['NUMBER OF CYCLIST INJURED'])

In [21]:
# Calculate weekly rate per intersection
weekly_intersection_stats = gdf.groupby(['year_week', 'intersection_id'])['total_casualties'].sum().reset_index()
weekly_intersection_stats.rename(columns={'total_casualties': 'weekly_intersection_rate'}, inplace=True)


In [22]:
# Merge back to original data
gdf = gdf.merge(weekly_intersection_stats, on=['year_week', 'intersection_id'], how='left')

In [25]:
# Check the results
print("New columns created:")
print(gdf[['year_week', 'intersection_id', 'total_casualties', 'weekly_intersection_rate']].head())

New columns created:
  year_week   intersection_id  total_casualties  weekly_intersection_rate
0  2021-W50  40.6231_-73.9581                 1                         1
1  2021-W50  40.6875_-73.9775                 1                         1
2  2022-W15  40.6501_-73.9302                 1                         1
3  2021-W49   40.5921_-73.963                 1                         1
4  2021-W49  40.6408_-73.9897                 1                         1


In [26]:
# Basic min/max
print("Weekly intersection rate range:")
print(f"Minimum: {gdf['weekly_intersection_rate'].min()}")
print(f"Maximum: {gdf['weekly_intersection_rate'].max()}")

# More detailed statistics
print("\nDetailed statistics:")
print(gdf['weekly_intersection_rate'].describe())

# See the actual records with min and max values
print("\nRow with minimum rate:")
print(gdf[gdf['weekly_intersection_rate'] == gdf['weekly_intersection_rate'].min()][['intersection_id', 'year_week', 'weekly_intersection_rate']].head(1))

print("\nRow with maximum rate:")
print(gdf[gdf['weekly_intersection_rate'] == gdf['weekly_intersection_rate'].max()][['intersection_id', 'year_week', 'weekly_intersection_rate']].head(1))

# Check value counts to see distribution
print("\nValue counts (top 10):")
print(gdf['weekly_intersection_rate'].value_counts().head(10))

Weekly intersection rate range:
Minimum: 1
Maximum: 9

Detailed statistics:
count    50493.000000
mean         1.035114
std          0.220491
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          9.000000
Name: weekly_intersection_rate, dtype: float64

Row with minimum rate:
    intersection_id year_week  weekly_intersection_rate
0  40.6231_-73.9581  2021-W50                         1

Row with maximum rate:
        intersection_id year_week  weekly_intersection_rate
42811  40.6517_-73.9304  2013-W12                         9

Value counts (top 10):
weekly_intersection_rate
1    48979
2     1313
3      166
4       28
8        3
6        1
5        1
7        1
9        1
Name: count, dtype: int64


#### Export file

In [28]:
os.makedirs('Clean Data', exist_ok=True)
gdf.to_file('brooklyn_ped_cyclist_crashes_2012_2024_WIR.geojson', driver='GeoJSON')